# AI Lesson 04b Part 2: GROUP BY and HAVING

**Course:** Applications of Artificial Intelligence  
**Focus:** Grouping Data and Filtering Groups  

---

## What You'll Learn Today

In Part 1, you learned aggregate functions (COUNT, SUM, AVG, MAX, MIN). Now you'll learn to **group data by categories**!

**Today's focus:**
- `GROUP BY` - Aggregate by categories
- `HAVING` - Filter grouped results
- Export grouped data to Excel
- Create team performance summaries

**Why this matters for ML:**
- Create features by team, season, or player
- Calculate win percentages per team
- Identify high-performing groups
- Prepare aggregated datasets for training

---

## Setup

In [1]:
# Import libraries
import pandas as pd
import sqlite3
import openpyxl

# Connect to database
conn = sqlite3.connect('nba_5seasons.db')

print("✅ Connected to NBA database")
print("Ready to learn GROUP BY!")

ModuleNotFoundError: No module named 'pandas'

---
## Part 1: Introduction to GROUP BY

GROUP BY lets you **aggregate data by categories**.

Example: Instead of "total points for all teams", get "total points for EACH team"

### Query 1: Count Games Per Season

How many games happened in EACH season?

**Pattern:**
```sql
SELECT category_column, COUNT(*)
FROM table
GROUP BY category_column
```

**Fill in the blanks:**

In [ ]:
# Count games per season
# GROUP BY season creates separate counts for each season
# Without GROUP BY, COUNT(*) would give total across all seasons
# With GROUP BY, we get one count per season
query = """
SELECT season, COUNT(*) as game_count
FROM team_game_stats
GROUP BY season
"""

# Run query and display results
result = pd.read_sql(query, conn)
display(result)

**Hint:** `GROUP BY season`

**What you should see:** One row per season with its game count

### Query 2: Count Wins and Losses

How many wins vs losses are in the database?

**Fill in the blank:**

In [ ]:
# Count by win/loss
# GROUP BY wl creates two groups: 'W' and 'L'
# Then COUNT(*) is calculated separately for each group
query = """
SELECT wl, COUNT(*) as count
FROM team_game_stats
GROUP BY ______
"""

result = pd.read_sql(query, conn)
display(result)

**Hint:** `GROUP BY wl`

**What you should see:** Two rows (W and L) with equal counts

---
## Part 2: GROUP BY with Different Aggregations

### Query 3: Average Points Per Season

Calculate average points for each season.

In [ ]:
# Average points grouped by season
# AVG(pts) is calculated separately for each season
# This shows if scoring has changed over the years
query = """
SELECT season, AVG(pts) as avg_points
FROM team_game_stats
GROUP BY ______
ORDER BY season
"""

result = pd.read_sql(query, conn)
display(result)

**Hint:** `GROUP BY season`

**Note:** ORDER BY season shows results chronologically!

### Query 4: Total Points Per Season

Sum up total points scored in each season.

In [ ]:
# Sum points per season
# SUM(pts) adds up all points for each season separately
# Shows total offensive output per season
query = """
SELECT season, SUM(______) as total_points
FROM team_game_stats
GROUP BY season
ORDER BY season
"""

result = pd.read_sql(query, conn)
display(result)

**Hint:** `SUM(pts)`

### Query 5: Max and Min Points Per Season

Find highest and lowest scores in each season.

In [ ]:
# Multiple aggregations with GROUP BY
# Each aggregate function works independently on each group
# Shows scoring range for each season
query = """
SELECT 
    season,
    MAX(pts) as highest_score,
    MIN(______) as lowest_score
FROM team_game_stats
GROUP BY season
ORDER BY season
"""

result = pd.read_sql(query, conn)
display(result)

**Hint:** `MIN(pts)`

---
## Part 3: Team Performance Analysis

### Query 6: Games Per Team (2021-22)

Count how many games each team played in 2021-22.

In [ ]:
# Count games by team_id
# Each team should have 82 games in a full season
# GROUP BY team_id creates one row per team
query = """
SELECT team_id, COUNT(*) as games_played
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY ______
ORDER BY games_played DESC
"""

result = pd.read_sql(query, conn)
display(result.head(10))

**Hint:** `GROUP BY team_id`

### Query 7: Average Points Per Team

Calculate each team's average points per game (2021-22).

In [ ]:
# Team offensive averages
# AVG(pts) per team shows which teams score more
# This is a key feature for predicting team success
query = """
SELECT 
    team_id,
    AVG(pts) as avg_points_per_game
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY team_id
ORDER BY ______ ______
LIMIT 10
"""

result = pd.read_sql(query, conn)
print("Top 10 scoring teams:")
display(result)

**Hint:** `ORDER BY avg_points_per_game DESC` (highest scorers first)

### Query 8: Comprehensive Team Stats

Get multiple stats for each team in one query!

In [ ]:
# Multiple aggregations per team
# Creates a team performance summary
# Perfect for ML features: avg_pts, avg_ast, avg_reb per team
query = """
SELECT 
    team_id,
    COUNT(*) as games,
    AVG(pts) as avg_pts,
    AVG(ast) as avg_ast,
    AVG(______) as avg_reb
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY team_id
ORDER BY avg_pts DESC
LIMIT 10
"""

result = pd.read_sql(query, conn)
print("Team performance summary:")
display(result)

**Hint:** `AVG(reb)` for rebounds

---
## Part 4: GROUP BY with Multiple Columns

### Query 9: Wins/Losses Per Team

Count wins AND losses for each team (2021-22).

**You can GROUP BY multiple columns!**

In [ ]:
# Group by team AND win/loss
# Creates groups for: (team1, W), (team1, L), (team2, W), (team2, L), etc.
# Shows each team's win-loss record
query = """
SELECT 
    team_id,
    wl,
    COUNT(*) as count
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY ______, ______
ORDER BY team_id, wl
"""

result = pd.read_sql(query, conn)
display(result.head(20))

**Hint:** `GROUP BY team_id, wl`

**What you should see:** Each team has 2 rows (one for W, one for L)

### Query 10: Season and Win/Loss Stats

Get average points by season AND win/loss.

In [ ]:
# Group by season and wl
# Shows: "In 2021-22, wins averaged X points, losses averaged Y"
# Helps understand scoring thresholds for winning
query = """
SELECT 
    season,
    wl,
    AVG(pts) as avg_points
FROM team_game_stats
GROUP BY season, wl
ORDER BY ______, ______
"""

result = pd.read_sql(query, conn)
display(result)

**Hint:** `ORDER BY season, wl` (show chronologically, W before L)

---
## Part 5: HAVING - Filtering Groups

WHERE filters rows BEFORE grouping.  
HAVING filters groups AFTER aggregating.

**Pattern:**
```sql
SELECT column, COUNT(*)
FROM table
WHERE row_condition
GROUP BY column
HAVING aggregate_condition
```

### Query 11: Teams with 40+ Wins

Find teams that won 40 or more games (2021-22).

In [ ]:
# HAVING filters the grouped results
# WHERE wl='W' filters rows before grouping
# HAVING COUNT(*)>=40 filters groups after counting
# Result: only teams with 40+ wins
query = """
SELECT 
    team_id,
    COUNT(*) as wins
FROM team_game_stats
WHERE season = '2021-22' AND wl = 'W'
GROUP BY team_id
HAVING COUNT(*) >= ______
ORDER BY wins DESC
"""

result = pd.read_sql(query, conn)
print("Teams with 40+ wins:")
display(result)

**Hint:** `HAVING COUNT(*) >= 40`

### Query 12: High-Scoring Teams

Find teams averaging 110+ points per game (2021-22).

In [ ]:
# HAVING with AVG
# Filters to only teams with high offensive averages
# These are the top offensive teams
query = """
SELECT 
    team_id,
    AVG(pts) as avg_points,
    COUNT(*) as games
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY team_id
HAVING AVG(pts) >= ______
ORDER BY avg_points DESC
"""

result = pd.read_sql(query, conn)
print("Teams averaging 110+ points:")
display(result)

**Hint:** `HAVING AVG(pts) >= 110`

### Query 13: Assist Leaders

Teams averaging 25+ assists per game (2021-22).

In [ ]:
# HAVING filters to high-assist teams
# Assists indicate good ball movement and teamwork
query = """
SELECT 
    team_id,
    AVG(ast) as avg_assists
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY ______
HAVING AVG(______) >= 25
ORDER BY avg_assists DESC
"""

result = pd.read_sql(query, conn)
print("Teams with great ball movement:")
display(result)

**Hint:** `GROUP BY team_id` and `HAVING AVG(ast) >= 25`

---
## Part 6: Player Statistics by Season

### Query 14: Players Who Played Multiple Seasons

Find players who appear in more than one season.

In [ ]:
# Count seasons per player
# GROUP BY player_id, then HAVING to filter
# Shows player longevity
query = """
SELECT 
    player_id,
    COUNT(DISTINCT season) as seasons_played
FROM player_season_stats
GROUP BY ______
HAVING COUNT(DISTINCT season) > 1
ORDER BY seasons_played DESC
LIMIT 20
"""

result = pd.read_sql(query, conn)
print("Players with multiple seasons:")
display(result)

**Hint:** `GROUP BY player_id`

**Note:** COUNT(DISTINCT season) counts unique seasons!

### Query 15: Average Stats Per Player

Get each player's average stats across ALL their seasons.

In [ ]:
# Career averages per player
# AVG of season averages = career average
# Only include players with 2+ seasons for meaningful career stats
query = """
SELECT 
    player_id,
    COUNT(*) as seasons,
    AVG(pts) as career_avg_pts,
    AVG(reb) as career_avg_reb,
    AVG(______) as career_avg_ast
FROM player_season_stats
GROUP BY player_id
HAVING COUNT(*) >= 2
ORDER BY career_avg_pts DESC
LIMIT 20
"""

result = pd.read_sql(query, conn)
print("Top career scorers (2+ seasons):")
display(result)

**Hint:** `AVG(ast)`

---
## Part 7: Excel Exports for Analysis

### 📊 Export 1: Team Performance Summary

In [ ]:
# Comprehensive team stats for 2021-22
# Perfect for Excel analysis, charts, PivotTables
query = """
SELECT 
    team_id,
    COUNT(*) as games_played,
    SUM(CASE WHEN wl = 'W' THEN 1 ELSE 0 END) as wins,
    SUM(CASE WHEN wl = 'L' THEN 1 ELSE 0 END) as losses,
    AVG(pts) as avg_pts,
    AVG(reb) as avg_reb,
    AVG(ast) as avg_ast,
    AVG(stl) as avg_stl,
    AVG(blk) as avg_blk,
    AVG(tov) as avg_tov
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY team_id
ORDER BY wins DESC
"""

# Get the data
team_summary = pd.read_sql(query, conn)

# Export to Excel
team_summary.to_excel('team_performance_2021-22.xlsx', 
                      index=False, 
                      sheet_name='Team Stats')

print(f"✅ Exported {len(team_summary)} teams to 'team_performance_2021-22.xlsx'")
print("\nPreview:")
display(team_summary.head(10))

**Note:** CASE WHEN is a way to count wins and losses separately!

### Export 2: Season Comparison

In [ ]:
# Stats by season - shows trends over time
# Great for creating line charts in Excel
query = """
SELECT 
    season,
    COUNT(*) as total_games,
    AVG(pts) as avg_points,
    AVG(fg3m) as avg_three_pointers,
    AVG(ast) as avg_assists,
    AVG(reb) as avg_rebounds,
    MAX(pts) as highest_score,
    MIN(pts) as lowest_score
FROM team_game_stats
GROUP BY ______
ORDER BY season
"""

# Get the data
season_comparison = pd.read_sql(query, conn)

# Export to Excel
season_comparison.to_excel(______, 
                           index=False, 
                           sheet_name='Season Trends')

print(f"✅ Exported {len(season_comparison)} seasons to 'season_trends.xlsx'")
display(season_comparison)

**Hint:** `GROUP BY season` and filename `'season_trends.xlsx'`

### Export 3: Win Probability Features

Create ML-ready dataset: team averages for predicting wins.

In [ ]:
# Team stats grouped by team and win/loss
# Shows average stats in wins vs losses
# Key ML features for predicting game outcomes
query = """
SELECT 
    team_id,
    wl,
    COUNT(*) as games,
    AVG(pts) as avg_pts,
    AVG(fgm) as avg_fgm,
    AVG(fg3m) as avg_fg3m,
    AVG(ftm) as avg_ftm,
    AVG(reb) as avg_reb,
    AVG(ast) as avg_ast,
    AVG(stl) as avg_stl,
    AVG(blk) as avg_blk,
    AVG(tov) as avg_tov
FROM team_game_stats
WHERE season = '2021-22'
GROUP BY team_id, wl
ORDER BY team_id, wl
"""

win_features = pd.read_sql(query, conn)

# Export to Excel AND CSV
win_features.to_excel('team_win_loss_features.xlsx', index=False, sheet_name='Features')
win_features.to_csv('team_win_loss_features.csv', index=False)

print("✅ Exported win/loss features to Excel and CSV")
print(f"   {len(win_features)} rows (each team has W and L stats)")
display(win_features.head(10))

---
## Part 8: Summary Query

Put it all together!

### Query 16: Complete Performance Analysis

Get comprehensive stats for playoff-caliber teams (45+ wins).

In [ ]:
# Complete analysis combining everything learned:
# - WHERE to filter season
# - GROUP BY team_id and wl
# - Multiple aggregations
# - HAVING to filter high-win teams
# - ORDER BY for ranking
query = """
SELECT 
    team_id,
    COUNT(*) as wins,
    AVG(pts) as avg_pts,
    AVG(ast) as avg_ast,
    AVG(reb) as avg_reb,
    AVG(tov) as avg_tov,
    MAX(pts) as best_game
FROM team_game_stats
WHERE season = '2021-22' AND wl = ______
GROUP BY team_id
HAVING ______ >= 45
ORDER BY wins DESC
"""

result = pd.read_sql(query, conn)
print("Playoff-caliber teams (45+ wins):")
display(result)

**Hint:** `wl = 'W'` and `HAVING COUNT(*) >= 45`

---
## Cleanup

In [ ]:
# Always close the connection
conn.close()
print("✅ Database connection closed")

---
## Congratulations!

You mastered GROUP BY and HAVING! You can now:

✅ **GROUP BY** single columns  
✅ **GROUP BY** multiple columns  
✅ Use aggregations with GROUP BY  
✅ **HAVING** to filter grouped results  
✅ **ORDER BY** grouped data  
✅ Export grouped data to Excel  
✅ Create ML-ready aggregated features  

---

## Key Takeaways

**GROUP BY:**
- Splits data into groups based on column values
- Aggregations run separately on each group
- Can group by multiple columns

**HAVING vs WHERE:**
- WHERE filters rows BEFORE grouping
- HAVING filters groups AFTER aggregating
- HAVING uses aggregate functions (COUNT, AVG, etc.)

**Clause Order:**
```sql
SELECT
FROM
WHERE
GROUP BY
HAVING
ORDER BY
LIMIT
```

---

## Excel Files Created

1. **team_performance_2021-22.xlsx** - Complete team stats
2. **season_trends.xlsx** - Stats by season
3. **team_win_loss_features.xlsx** - ML features

**Use these for:**
- Creating charts and visualizations
- Building PivotTables
- MOS Excel practice
- Presenting insights to stakeholders

---

## Next Steps

You're ready for the next lessons where you'll:
- Build actual ML models with your SQL data
- Create training/test datasets
- Predict game outcomes
- Evaluate model performance